# Ensemble Attack Implementation

TJ Kim, 11.16.21

#### Summary:
- Build upon the "Transferer" class that performs adversarial perturbations based on gradients of one model and sends adversarial examples to other members of ensemble
- Ensemble Cascade Adversarial Attack using multiple members of ensembles as adversarial

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


#### Import Libraries

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

#### Build New Ensemble Transferer Subclass

In [3]:
class Ensemble_Transferer(Transferer): 
    def __init__(self, models_list, dataloader):
        super(Ensemble_Transferer, self).__init__(models_list, dataloader)
    
        # Other Params
        self.advNN_idx = None # list
        self.advNN = None # dict of personalized_NN
        self.atk_order = None
        
    def generate_advNN(self, client_idx):
        """
        Select specific client to load neural network to 
        Load the data for that client
        Lod the weights for that client
        This is the client that will generate perturbations
        """        
        # Import the loader for this dataset only
        
        self.advNN_idx = client_idx # List
        self.advNN = {} # Dict of NN
        
        for i in client_idx:
            self.advNN[i] = copy.deepcopy(Adv_NN(self.models_list[i], self.dataloader))
        
        return
    
    def choose_attack_sequence(self):
        """
        Given the number of iterations of attack in params, and number of adv nn
        choose a list of len~iter of which idx to attack
        must run generate_advNN prior 
        """
    
        num_iters = self.ifsgm_params.iteration
        atk_order = []
        
        for t in range(num_iters):
            idx = t%len(self.advNN_idx)
            atk_order += [self.advNN_idx[idx]]
            
        self.atk_order = atk_order
        
    
    def generate_xadv(self, atk_type = "IFSGM", mode='test'):
        """
        Generate perturbed images
        atk_type - "IFSGM"
        """
        
        # Import attack parameters
        batch_size = self.ifsgm_params.batch_size
        
        # Load data to perturb
        data_x, data_y = self.dataloader.load_batch(batch_size, mode=mode)
        self.x_orig  = data_x.reshape(batch_size,3,32,32)
        self.y_true = data_y.type(torch.LongTensor)
        self.y_orig = self.y_true # Unused dummy feature
        self.y_adv = self.y_true
        
        x_in = copy.deepcopy(self.x_orig)
        y_in  = self.y_true
        
        # Alter number of iteration in params to 1 
        temp_params = copy.deepcopy(self.ifsgm_params)
        temp_params.iteration = 1
        
        for idx in self.atk_order:
            self.advNN[idx].i_fgsm_sub(temp_params, x_in, y_in)
            x_in = copy.deepcopy(self.advNN[idx].x_adv)
        
        # Record relevant tensors
        self.x_adv = copy.deepcopy(x_in).cuda()
        
    def send_to_victims(self, client_idxs):
        """
        Send pre-generated adversarial perturbations 
        client_idxs - list of indices of clients we want to attack (just victims)
        
        Then record the attack success stats accordingly
        """
        
        for i in client_idxs:
            self.victims[i].ensemble_forward_transfer(self.x_orig, self.x_adv, self.y_orig, self.ifsgm_params.target)

            # Record Performance
            self.orig_target_hit[i] = self.victims[i].orig_target_achieve
            self.adv_target_hit[i] = self.victims[i].adv_target_achieve

### FedEM Analysis 
- Generate aggregator and extract saved models

In [36]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedEM"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_09_28_first_transfers/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 253.28it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:27<00:00,  2.87it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.159% |Test Loss: 2.292 | Test Acc: 12.248% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [37]:
# IMPORT DATASET -- DON'T IMPORT FOR FED LOCAL CASE LATER

# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

In [38]:
# Import weights for aggregator
aggregator.load_state(args_.save_path)

# This is where the models are stored -- one for each mixture --> learner.model for nn
hypotheses = aggregator.global_learners_ensemble.learners

# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]

In [39]:
weights = np.load("weights/cifar/21_09_28_first_transfers/train_client_weights.npy")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

#print(weights)

# Set model weights
model_weights = []
num_models = 7

for i in range(num_models):
    model_weights += [weights[i]]
    
    
# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for (w0,w1,w2) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]

### Set up Transfer Attack Scenario

In [40]:
# Set Up Dictionaries -- list holds the adversary idx
logs_adv = []

for i in range(len(model_weights)):
    adv_dict = {}
    adv_dict['orig_target_hit'] = None
    adv_dict['adv_target_hit'] = None
    logs_adv += [adv_dict]

In [41]:
adv_idx = [0,1,2]
victim_idxs = [0,1,2,3,4,5,6]

t1 = Ensemble_Transferer(models_list=models_test, dataloader=dataloader)

# atk Params setup
t1.ifsgm_params.set_params(batch_size=500, eps=0.1, alpha=0.05, iteration = 30,
               target = 9, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x))

t1.generate_victims(victim_idxs)
t1.generate_advNN(adv_idx)
t1.advNN_idx = adv_idx
t1.choose_attack_sequence()
t1.generate_xadv(atk_type = "ifsgm")
t1.send_to_victims(victim_idxs)

# Log Performance
# logs_adv[adv_idx]['orig_target_hit'] = t1.orig_target_hit
# logs_adv[adv_idx]['adv_target_hit'] = t1.adv_target_hit

In [42]:
t1.orig_target_hit

{0: tensor(0.0940, device='cuda:0'),
 1: tensor(0.1180, device='cuda:0'),
 2: tensor(0.0840, device='cuda:0'),
 3: tensor(0.0840, device='cuda:0'),
 4: tensor(0.0820, device='cuda:0'),
 5: tensor(0.1120, device='cuda:0'),
 6: tensor(0.1060, device='cuda:0')}

In [43]:
t1.adv_target_hit

{0: tensor(1., device='cuda:0'),
 1: tensor(1., device='cuda:0'),
 2: tensor(0.9980, device='cuda:0'),
 3: tensor(0.9980, device='cuda:0'),
 4: tensor(0.9980, device='cuda:0'),
 5: tensor(1., device='cuda:0'),
 6: tensor(1., device='cuda:0')}

### FedLocal Analysis

In [44]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "local"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 1
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_11_06_local/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 302.50it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:09<00:00,  8.21it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.299 | Train Acc: 10.643% |Test Loss: 2.298 | Test Acc: 10.503% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [45]:
# Import weights for aggregator
aggregator.load_state(args_.save_path)

# Set model weights
weights = np.load("weights/cifar/21_11_06_local/train_client_weights.npy")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

model_weights = []
num_models = 7

for i in range(num_models):
    model_weights += [weights[i]]

# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for i in range(num_models):
    new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
    new_model.eval()
    models_test += [new_model]

In [46]:
# Set Up Dictionaries -- list holds the adversary idx
logs_adv = []

for i in range(len(model_weights)):
    adv_dict = {}
    adv_dict['orig_target_hit'] = None
    adv_dict['adv_target_hit'] = None
    logs_adv += [adv_dict]

In [47]:
adv_idx = [0,1,2]
victim_idxs = [0,1,2,3,4,5,6]

t1 = Ensemble_Transferer(models_list=models_test, dataloader=dataloader)

# atk Params setup
t1.ifsgm_params.set_params(batch_size=500, eps=0.1, alpha=0.05, iteration = 30,
               target = 9, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x))

t1.generate_victims(victim_idxs)
t1.generate_advNN(adv_idx)
t1.advNN_idx = adv_idx
t1.choose_attack_sequence()
t1.generate_xadv(atk_type = "ifsgm")
t1.send_to_victims(victim_idxs)

# Log Performance
# logs_adv[adv_idx]['orig_target_hit'] = t1.orig_target_hit
# logs_adv[adv_idx]['adv_target_hit'] = t1.adv_target_hit

In [48]:
t1.orig_target_hit

{0: tensor(0.1060, device='cuda:0'),
 1: tensor(0.2520, device='cuda:0'),
 2: tensor(0.1380, device='cuda:0'),
 3: tensor(0.1300, device='cuda:0'),
 4: tensor(0.0240, device='cuda:0'),
 5: tensor(0.1340, device='cuda:0'),
 6: tensor(0.1800, device='cuda:0')}

In [49]:
t1.adv_target_hit

{0: tensor(0.9820, device='cuda:0'),
 1: tensor(0.9700, device='cuda:0'),
 2: tensor(0.9940, device='cuda:0'),
 3: tensor(0.5920, device='cuda:0'),
 4: tensor(0.2060, device='cuda:0'),
 5: tensor(0.8980, device='cuda:0'),
 6: tensor(0.8120, device='cuda:0')}